In [ ]:
#| default_exp routes.role

In [ ]:
#| export
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de


# Get and Search Routes

In [ ]:
#| export
class Role_NotRetrieved(de.DomoError):
    def __init__(self,
                 domo_instance,
                 function_name,
                 status,
                 message,
                 role_id = None,
                 ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=role_id,
            function_name=function_name,
            status=status,
            message=message
        )


In [ ]:
# | export
async def get_roles(auth: dmda.DomoAuth,
                    debug_api: bool = False,
                    session: httpx.AsyncClient = None,
                    ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles'

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if len(res.response) == 0 or not res.is_success:
        raise Role_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name='get_roles',
            status=res.status,
            message = 'failed to retrieve roles')
    
    return res


#### sample implementation of get_roles

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_roles(auth=token_auth)

pd.DataFrame(res.response[0:5])


,id,name,description,authorityCount,userCount,created,modified
0,1,Admin,Full access to everything,77,16,1580425515000,1580425515000
1,2,Privileged,Full access except for editing users and setti...,34,313,1580425515000,1580425515000
2,3,Editor,"Can edit Cards, Pages, DataSets, and Dataflows",30,4,1580425515000,1580425515000
3,4,Participant,"Read only access on Cards, Pages DataSets and ...",6,4,1580425515000,1580425515000
4,5,Social,Access limited to social features,1,11,1580425515000,1580425515000


In [ ]:
#| export


async def get_role_grants(auth: dmda.DomoAuth,
                          role_id: str,
                          debug_api: bool = False,
                          session : httpx.AsyncClient = None
                          ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/authorities'

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session = session
    )

    if len(res.response) == 0:

        role_res = await get_roles(auth = auth)

        domo_role = [ role for role in role_res.response if role.get('id') ==role_id]

        if not domo_role:
            raise Role_NotRetrieved(
                domo_instance= auth.domo_instance,
                function_name='get_role_grants',
                message=f"role {role_id} does not exist",
                status=res.status
            )

    return res

#### sample implementation of get_role_grants

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_role_grants(auth=token_auth, role_id = 1)

pd.DataFrame(res.response[0:5])

,0
0,versions.deployment.manage
1,buzz.admin
2,dataflow.admin
3,goal.admin
4,dataset.admin


In [ ]:
#| export
async def get_role_membership(auth: dmda.DomoAuth,
                              role_id: str,
                              debug_api: bool = False, 
                              session: httpx.AsyncClient = None,
                              return_raw : bool = False
                              ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/users'

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if len(res.response.get('users')) == 0:
        
        role_res = await get_roles(auth)

        domo_role = next((role for role in role_res.response if role.get('id') == role_id), None)

        if not domo_role:
            raise Role_NotRetrieved(
                domo_instance=auth.domo_instance,
                function_name='get_role_membership',
                message=f"role {role_id} does not exist or cannot be retrieved",
                status=res.status
            )

    if return_raw:
        return res
    
    res.response = res.response.get('users')
    
    return res

#### sample implementation of get_role_membership


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_role_membership(auth=token_auth, role_id=1, return_raw = False)

pd.DataFrame(res.response[0:5])


,userId,email,displayName,roleId,department,jobTitle,hireDate,location,timeZone
0,1345408759,DataMaven42@gmail.com,Alexis Lorenz (DataMaven),1,Owner,Major Domo Support Specialist,1.590156e+12,NaN,NaN
1,1102391116,andrea.henderson@domo.com,Andrea Henderson,1,NaN,NaN,NaN,NaN,NaN
2,1095515127,andrew@crystalballers.ai,Andrew Chaffin,1,Crystal Ballers,Cofounder,NaN,,NaN
3,2074422502,bard.higgins@gmail.com,Bard Higgins,1,NaN,NaN,NaN,NaN,America/Denver
4,1865099622,Ben.Schein@domo.com,Ben Schein,1,NaN,NaN,NaN,NaN,NaN


# CRUD Routes

In [ ]:
#| export
async def create_role(auth: dmda.DomoAuth,
                      name: str,
                      description: str,
                      debug_api: bool = False,
                      session :httpx.AsyncClient = None
                      ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles'

    body = {
        'name': name,
        'description': description
    }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        debug_api=debug_api,
        body=body,
        session = session
    )

    return res

In [ ]:
# | export
async def delete_role(auth: dmda.DomoAuth,
                      role_id: int,
                      debug_api: bool = False,
                      session: httpx.AsyncClient = None
                      ) -> rgd.ResponseGetData:
    
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}'

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='DELETE',
        debug_api=debug_api,
        session=session
    )

    if res.is_success:
        res.response = f"role {role_id} deleted or doesn't exist"

    return res


#### sample implementation of delete_role

In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await delete_role(auth=token_auth, role_id='2014090557')


ResponseGetData(status=204, response="role 2014090557 deleted or doesn't exist", is_success=True)

### Default Role

In [ ]:
#| export
async def get_default_role(auth, debug_api: bool = False, session :httpx.AsyncClient= None):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/user.roleid.default"

    params = {'defaultValue': 2, 'ignoreCache': True}

    res = await gd.get_data(auth=auth,
                             method='GET',
                             url=url,
                             params=params,
                             debug_api=debug_api, session = session)

    
    if res.is_success:
        res.response = res.response.get('value')
    
    return res



#### sample implementation of get_default_role

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await get_default_role(token_auth)

ResponseGetData(status=200, response='2097317660', is_success=True)

In [ ]:
# | export
async def set_default_role(auth: dmda.DomoAuth,
                           role_id: str,
                           debug_api: bool = False,
                           session: httpx.AsyncClient = None) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/user.roleid.default'

    body = {
        'name': 'user.roleid.default',
        'value': role_id
    }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        debug_api=debug_api,
        body=body,
        session=session
    )
    return res


### Metadata

In [ ]:
#| export
async def update_role_metadata(auth: dmda.DomoAuth,
                               role_id,
                               role_name,
                               role_description: str = None,
                               debug_api: bool = False,
                               session : httpx.AsyncClient = None
                               ):

    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}"

    body = {"name": role_name,
            "description": role_description, "id": role_id}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        debug_api=debug_api,
        body=body,
        session=session
    )

    
    return res


#### sample update_role_metadata

In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#get role to update
role_ls = await get_roles(auth = token_auth)
role = next((role for role in role_ls.response if role.get('name') == 'super_admin'),None)


# update role
await update_role_metadata(auth=token_auth,
                     role_id = role['id'],
                     role_name= role['name'],
                           role_description=f'last updated - {dt.date.today()}'
                     )

# validate role has been updated
role_ls = await get_roles(auth=token_auth)
role = next((role for role in role_ls.response if role.get(
    'name') == 'super_admin'), None)

role


{'id': 810756122,
 'name': 'super_admin',
 'description': 'last updated - 2023-03-20',
 'authorityCount': 84,
 'userCount': 0,
 'created': 1678842530000,
 'modified': 1679335795000}

In [ ]:
#| export
async def set_role_grants(auth: dmda.DomoAuth,
                             role_id: str,
                             role_grant_ls: list[str],
                             debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/authorities'

    if debug_api:
        print(url)

    res = await gd.get_data(auth=auth,
                         url=url,
                         method='PUT',
                         debug_api=debug_api, body=role_grant_ls)
    return res

In [ ]:
#| export
async def role_membership_add_users(auth: dmda.DomoAuth,
                                   role_id: str,
                                   user_list: list[str], # list of user ids
                                   debug_api: bool = False,
                                   session : httpx.AsyncClient = None
                                   ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/users'

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        debug_api=debug_api,
        body=user_list,
        session = session
    )
    return res

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()
